In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import os
import time
from HERMES import reference

LANGUAGES = [
	{"name":"German", "short":"d"},
    {"name":"French", "short":"f"}
]
language = LANGUAGES[0] #Change this to select language.

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [6]:
# --------------------------------------------------------------
# Assistants IDs
# --------------------------------------------------------------
#QAassistant = client.beta.assistants.retrieve("asst_otEHfkni1yg695z3lSFA8MHh")

QAassistant = client.beta.assistants.retrieve("asst_RF4gChYqyrrpMENDifCWfdmY")

# --------------------------------------------------------------
# Generation of answers using an Assistant
# --------------------------------------------------------------

def run_assistant(thread,assistant):

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,

    )

    # Wait for completion
    while run.status != "completed":
        # Be nice to the API
        time.sleep(0.2)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    # Retrieve the Messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    return messages.data[0].content[0].text.value

def generate_response(assistant,prompt):

    thread = client.beta.threads.create(
        messages = [
            {
            "role" : "user",
            "content" : prompt
        }]
    )

    response = run_assistant(thread,assistant)
    try:
        jsontest = json.loads(response)
    except ValueError as e:
        print("Response was not returned as valid JSON. Attempting to clear this up.")
        stripped = response.replace('```','').replace('json','')
        return stripped
    return response

In [19]:
with open(f"learning-objectives.json", 'r') as obfile:
	#for json file -> dictionary creation
	objectives = json.load(obfile)["reference"]

#return a single string of all relevant objectives, concatenated.
def get_objective(chapter):
    return "Learn well."
    """  parts = chapter.split(".") #
    objectives_list = []
    key=""
    for val in parts:
        key = val if len(key)<1 else key+"."+val
        obj = [o for o in objectives if key in o['chapters']]
        for o in obj:
            objectives_list.append(o[f"objective-{language['short']}"])
    return  ''.join(objectives_list) """


In [20]:
def generateQA(assistant,text,objective):
    prompt = f"""
    Your task is to generate a question and answer based on the following steps:
    Step 1: Based on the provided text: "{text}", generate a one sentence question in {language["name"]}.
    Step 2: Ensure the question:
    - Is clear
    - Is relevant
    - Is meaningful according to the objective: "{objective}"
    - Uses simple vocabulary
    - Asks about an important aspect of the text
    - Asks only one thing
    Step 3: Retrieve the correct answer by citing the exact sentence provided in the text.
    Step 4: Provide the output as valid JSON with the keys "Question" and "Answer".
    """

    response = generate_response(QAassistant,prompt)

    data = json.loads(response)
    question= data["Question"]
    answer = data["Answer"]
    return question,answer


In [62]:
reference = reference
# [

#  {"chapter": "7.4.8.3",

#      "information": '''
#        Die definierten HERMES-Szenarien und -Module können mit Ergebnissen und den dazu-
#     gehörigen Aufgaben erweitert, aber nicht reduziert werden.'''},


# ]

In [21]:
training_questions = []
short_reference = reference[0:5] #first five only, for testing
for ref in short_reference:
    information = ref['information']
    chapter = ref['chapter']
    objective = get_objective(chapter)
    response = generateQA(QAassistant,information,objectives)
    question_dict = {}
    question_dict["question_text"] = response[0]
    question_dict["correct_answer"] = response[1]
    question_dict["link"] = chapter
    training_questions.append(question_dict)

training_questions


[{'question_text': "Was ist der Inhalt des Textes mit dem Titel 'Methodenüberblick'?",
  'correct_answer': 'Methodenüberblick',
  'link': 'A'},
 {'question_text': 'Was versinnbildlichen die roten ovalen Iterationspfeile im Kontext von HERMES?',
  'correct_answer': 'Die roten ovalen Iterationspfeile versinnbildlichen den Kern der Iteration, den treibenden Charakter der Module Produkt und IT-System während der agilen Entwicklung.',
  'link': 'A.1'},
 {'question_text': 'Wie ist das HERMES-Projektmanagement strukturiert und welches sind die gleichwertigen Elemente dieser Methode?',
  'correct_answer': 'Wie die Abbildung 2 zeigt, sind HERMES-Portfoliomanagement, HERMES-Projektmanagement und HERMES-Anwendungsmanagement gleichwertige Methodenelemente und bilden gemeinsam die HERMES-Methode.',
  'link': 'A.2'},
 {'question_text': 'Wie wird im HERMES-Projektmanagement die Sizing-Funktion zur Anpassung der Projektszenarien verwendet?',
  'correct_answer': 'Mit der im HERMES-Online bereitgestellt

In [64]:

training_questions


[{'question_text': "Was verstehen die Teilnehmer unter dem Methodenelement 'Ergebnissen' und 'Aufgaben'?",
  'correct_answer': "Die Teilnehmer wissen, was unter Methodenelementen 'Ergebnissen' und 'Aufgaben' verstanden wird.【4:3†Objectives-2.txt】",
  'link': 'A'},
 {'question_text': 'Was symbolisieren die roten ovalen Iterationspfeile in der HERMES-Methode?',
  'correct_answer': 'Die roten ovalen Iterationspfeile versinnbildlichen den Kern der Iteration, den treibenden Charakter der Module Produkt und IT-System während der agilen Entwicklung.',
  'link': 'A.1'},
 {'question_text': 'Was unterstützt HERMES-Projektmanagement?',
  'correct_answer': 'HERMES-Projektmanagement unterstützt die Steuerung, Führung und Ausführung von Vorhaben und begleitet die Weiterentwicklung von organisatorischen Strukturen, Produkten und Dienstleistungen, IT- und Logistiksystemen, Infrastrukturen u. ä. unterschiedlicher Charakteristiken und Komplexitäten.',
  'link': 'A.2'},
 {'question_text': 'Wie passt die 